# day 5

https://adventofcode.com/5/day/5

In [1]:
import logging
import logging.config
import os

import yaml

In [2]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [3]:
FNAME = os.path.join('data', 'day05.txt')

LOGGER = logging.getLogger('day05')

## part 1

### problem statement:

#### loading data

In [4]:
test_data = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

In [5]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [19]:
def parse_data(s: str) -> tuple[dict[int, list[int]], list[list[int]]]:
    order_rules, update_pagelists = s.split('\n\n')
    order_rules_dict = {}
    for line in order_rules.split('\n'):
        a, b = [int(_) for _ in line.strip().split('|')]
        if a not in order_rules_dict:
            order_rules_dict[a] = []
        order_rules_dict[a].append(b)
    update_pagelists = [[int(_) for _ in line.strip().split(',')] for line in update_pagelists.split('\n')]
    return order_rules_dict, update_pagelists

In [20]:
parse_data(test_data)

({47: [53, 13, 61, 29],
  97: [13, 61, 47, 29, 53, 75],
  75: [29, 53, 47, 61, 13],
  61: [13, 53, 29],
  29: [13],
  53: [29, 13]},
 [[75, 47, 61, 53, 29],
  [97, 61, 53, 29, 13],
  [75, 29, 13],
  [75, 97, 47, 61, 53],
  [61, 13, 29],
  [97, 13, 75, 29, 47]])

#### function def

In [21]:
def get_middle_page(update_pagelist: list[int]) -> int:
    l = len(update_pagelist)
    return update_pagelist[l // 2]

assert get_middle_page(update_pagelist=[75, 47, 61, 53, 29]) == 61
assert get_middle_page(update_pagelist=[97, 61, 53, 29, 13]) == 53
assert get_middle_page(update_pagelist=[75, 29, 13]) == 29

In [25]:
def is_ordered(update_pagelist: list[int], order_rules: dict[int, list[int]]) -> bool:
    for (i, page) in enumerate(update_pagelist):
        try:
            rules = order_rules[page]
            if set(update_pagelist[:i]).intersection(rules):
                return False
        except KeyError:
            continue
    return True

order_rules, update_pagelists = parse_data(test_data)
assert is_ordered(update_pagelists[0], order_rules)
assert is_ordered(update_pagelists[1], order_rules)
assert is_ordered(update_pagelists[2], order_rules)
assert not is_ordered(update_pagelists[3], order_rules)
assert not is_ordered(update_pagelists[4], order_rules)
assert not is_ordered(update_pagelists[5], order_rules)

In [27]:
def q_1(data):
    order_rules, update_pagelists = parse_data(data)
    return sum([get_middle_page(update_pagelist=update_pagelist)
                for update_pagelist in update_pagelists
                if is_ordered(update_pagelist=update_pagelist, order_rules=order_rules)])

#### tests

In [28]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) ==143
    LOGGER.setLevel(logging.INFO)

In [29]:
test_q_1()

#### answer

In [30]:
q_1(load_data())

4790

## part 2

### problem statement:

#### function def

In [47]:
class Page:
    def __init__(self, value: int, order_rules: dict[int, list[int]]):
        self.value = value
        self.order_rules = order_rules

    def __lt__(self, other) -> bool:
        a = self.value
        b = other.value
        return b in self.order_rules.get(a, [])

    def __repr__(self):
        return f"Page({self.value})"

In [48]:
order_rules, update_pagelists = parse_data(test_data)

assert Page(47, order_rules) < Page(53, order_rules)
assert not Page(53, order_rules) < Page(47, order_rules)

In [49]:
def sort_update_pagelist(update_pagelist: list[int], order_rules: dict[int, list[int]]) -> list[int]:
    p = [Page(value=_, order_rules=order_rules) for _ in update_pagelist]
    p = sorted(p)
    return [_.value for _ in p]

def is_ordered(update_pagelist: list[int], order_rules: dict[int, list[int]]) -> bool:
    return sort_update_pagelist(update_pagelist, order_rules) == update_pagelist

order_rules, update_pagelists = parse_data(test_data)
assert is_ordered(update_pagelists[0], order_rules)
assert is_ordered(update_pagelists[1], order_rules)
assert is_ordered(update_pagelists[2], order_rules)
assert not is_ordered(update_pagelists[3], order_rules)
assert not is_ordered(update_pagelists[4], order_rules)
assert not is_ordered(update_pagelists[5], order_rules)

In [50]:
def q_2(data):
    order_rules, update_pagelists = parse_data(data)
    return sum([get_middle_page(sort_update_pagelist(update_pagelist=update_pagelist, order_rules=order_rules))
                for update_pagelist in update_pagelists
                if not is_ordered(update_pagelist=update_pagelist, order_rules=order_rules)])

#### tests

In [51]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 123
    LOGGER.setLevel(logging.INFO)

In [52]:
test_q_2()

#### answer

In [53]:
q_2(load_data())

6319

fin